In [ ]:
import os

import gcsfs
import pandas as pd
import wandb

from tqdm.auto import tqdm


In [ ]:
exchange = "ise"
strategy = "unsupervised"
max_i = 30  # number of partial files


In [ ]:
# connect to weights and biases
run = wandb.init(project="thesis", job_type="dataset-creation", entity="fbv")
dataset = wandb.Artifact(name=f"{exchange}_{strategy}_raw", type="preprocessed_data")


In [ ]:
os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

fs = gcsfs.GCSFileSystem(project="thesis")


In [ ]:
files = [
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/{'matched' if strategy == 'supervised' else 'unmatched'}_{exchange}_quotes_min_mem_usage_extended_part_{i:04d}.parquet"
    for i in range(0, max_i)
]

columns = [
    "QUOTE_DATETIME",
    "ROOT",
    "EXPIRATION",
    "STRK_PRC",
    "OPTION_TYPE",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "BEST_BID",
    "BEST_ASK",
    "ask_ex",
    "bid_ex",
    "bid_size_ex",
    "ask_size_ex",
    "price_all_lead",
    "price_all_lag",
    "optionid",
    "price_ex_lead",
    "price_ex_lag",
    "issue_type",
    "myn",
]

dfs = [pd.read_parquet(gc_file, columns=columns) for gc_file in tqdm(files)]
df = pd.concat(dfs)

del dfs


In [ ]:
df.memory_usage(deep=True).sum()


In [ ]:
df.sort_values(by="QUOTE_DATETIME", inplace=True)


In [ ]:
df.head()


In [ ]:
columns = ["buy_sell", *df.columns.tolist()]
labelled_df = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/{exchange}_supervised_train.parquet",
    columns=columns,
)


In [ ]:
labelled_df.head()


In [ ]:
# slice labelled dataframe to range of unlabelled
date_range = labelled_df.QUOTE_DATETIME.between(
    df.QUOTE_DATETIME.min(), df.QUOTE_DATETIME.max()
)
labelled_df = labelled_df[date_range]


In [ ]:
subset = [
    "QUOTE_DATETIME",
    "ROOT",
    "OPTION_TYPE",
    "EXPIRATION",
    "STRK_PRC",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "BEST_BID",
    "BEST_ASK",
    "ask_ex",
    "bid_ex",
    "bid_size_ex",
    "ask_size_ex",
    # Can be different for lobelled and unlabelled trades:
    # 'optionid','issue_type', 'myn',
    # 'price_all_lead', 'price_all_lag',
    # 'price_ex_lead', 'price_ex_lag',
]

labelled_df["duplicated"] = labelled_df.duplicated(subset=subset)


In [ ]:
labelled_df["duplicated"].value_counts()


In [ ]:
labelled_df[labelled_df["optionid"] == 83414152.0].head(20).T


In [ ]:
labelled_df["index_labelled"] = labelled_df.index


In [ ]:
len_labelled_df = len(labelled_df)
len_df = len(df)


In [ ]:
# merge with indicator and find out which ones can be labelled
df_w_indicator = pd.merge(
    df,
    labelled_df,
    on=[
        "QUOTE_DATETIME",
        "ROOT",
        "OPTION_TYPE",
        "EXPIRATION",
        "STRK_PRC",
        "TRADE_SIZE",
        "TRADE_PRICE",
        "BEST_BID",
        "BEST_ASK",
        "ask_ex",
        "bid_ex",
        "bid_size_ex",
        "ask_size_ex",
        # myn seems to be different for labelled and unlabelled trades
        # 'issue_type', 'optionid',
        # 'price_all_lead', 'price_all_lag',
        # 'price_ex_lead', 'price_ex_lag',
    ],
    how="left",
    indicator="exists",
    suffixes=("_unlabelled", "_labelled"),
)


In [ ]:
df_w_indicator.head(50)


In [ ]:
# interpolate missing indices. index increases 1 -> 2. So filling with float seems ok. will be inserted between int of labelled df.
df_w_indicator["index_labelled"].interpolate("linear", inplace=True)
df_w_indicator.set_index(keys="index_labelled", drop=True, inplace=True)


In [ ]:
df_w_indicator.head()


In [ ]:
len(df_w_indicator)


In [ ]:
len(df)


In [ ]:
len(labelled_df)


In [ ]:
# sort columns lexigraphically
df_w_indicator.sort_index(axis=1, inplace=True)


In [ ]:
df_w_indicator[df_w_indicator["exists"] == "both"].head(20).T


## Creation of Unsupervised Sample

In [ ]:
# use last 6 months. May increase later
date_range = df_w_indicator.QUOTE_DATETIME.between(
    "2013-04-24 00:00:00", "2013-10-24 16:14:48"
)

df_w_indicator = df_w_indicator[date_range]


In [ ]:
df_w_indicator.head(5).T


In [ ]:
# add fields
df_w_indicator["buy_sell"] = 0


In [ ]:
# create day_vol of option series. Note optionid is not enough for option series. Use key (p. 8) from paper instead.
df_w_indicator["day_vol"] = df_w_indicator.groupby(
    [
        df_w_indicator["QUOTE_DATETIME"].dt.date,
        df_w_indicator["OPTION_TYPE"],
        df_w_indicator["ROOT"],
        df_w_indicator["STRK_PRC"],
        df_w_indicator["EXPIRATION"],
    ]
)["TRADE_SIZE"].transform("sum")


In [ ]:
df_w_indicator


In [ ]:
train = df_w_indicator[df_w_indicator["exists"] == "left_only"].drop(
    columns=[
        "issue_type_labelled",
        "myn_labelled",
        "optionid_labelled",
        "price_all_lag_labelled",
        "price_all_lead_labelled",
        "price_ex_lag_labelled",
        "price_ex_lead_labelled",
        "exists",
        "duplicated",
    ]
)
train.columns = train.columns.str.replace(r"_unlabelled$", "", regex=True)


In [ ]:
train.head().T


In [ ]:
train.describe()


In [ ]:
output_path = f"gs://thesis-bucket-option-trade-classification/data/preprocessed/{exchange}_{strategy}_train.parquet"
train.to_parquet(output_path)
dataset.add_reference(output_path, name="train_set")


In [ ]:
# Log the artifact to save it as an output of this run
run.log_artifact(dataset)
wandb.finish()
